In [1]:
import json
from langchain_core.documents import Document

def load_json_to_documents(filepath):
    with open(filepath, 'r') as f:
        data = json.load(f)

    docs = []
    for entry in data:
        content = entry.get("content", "")
        metadata = {k: v for k, v in entry.items() if k != "content"}
        docs.append(Document(page_content=content, metadata=metadata))
    return docs

docs = load_json_to_documents("data.json")

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma

# Split into smaller chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
split_docs = splitter.split_documents(docs)

# Use Ollama for embeddings (make sure Ollama is running)
embedding = OllamaEmbeddings(model="gemma:2b")

# Create vector DB
vectordb = Chroma.from_documents(split_docs, embedding=embedding)
retriever = vectordb.as_retriever()

C:\Users\karth\AppData\Local\Temp\ipykernel_22088\1138914918.py:10: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedding = OllamaEmbeddings(model="gemma:2b")


In [6]:
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

#from langchain.chains import create_stuff_documents_chain, create_retrieval_chain
from langchain_community.llms import Ollama

# Define your prompt
prompt = PromptTemplate.from_template("""
You are an expert assistant. Use the below context to answer the question.

{context}

Question: {input}
Answer:""")

# Connect LLM
llm = Ollama(model="gemma:2b")

# Create "stuff" chain to combine retrieved docs
doc_chain = create_stuff_documents_chain(llm=llm, prompt=prompt)

# Full RAG chain
rag_chain = create_retrieval_chain(retriever=retriever, combine_docs_chain=doc_chain)

In [7]:
question = "What is Ollama?"
response = rag_chain.invoke({"input":question})
print("Answer:", response)

Answer: {'input': 'What is Ollama?', 'context': [Document(metadata={'title': 'What is Ollama?'}, page_content='Ollama allows running large language models locally like LLaMA and Mistral.'), Document(metadata={'title': 'LangChain Overview'}, page_content='LangChain is a framework for developing applications powered by language models.'), Document(metadata={'title': 'Embeddings in NLP'}, page_content='Embeddings are vector representations of text used for similarity and retrieval.')], 'answer': 'The context does not provide information about what Ollama is, so I cannot answer this question from the provided context.'}
